In [7]:
import pandas as pd
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
from folium.plugins import TimeSliderChoropleth


In [8]:
# Leggo il file di shapes

comuni_shape = gpd.read_file('shapes/Com01012021_g_WGS84.shp',encoding='utf-8')[['COMUNE','geometry']]
comuni_shape

,COMUNE,geometry
0,Agliè,"POLYGON ((404703.558 5026682.655, 405359.554 5..."
1,Airasca,"POLYGON ((380700.909 4977305.520, 381105.998 4..."
2,Ala di Stura,"POLYGON ((364710.857 5022090.678, 364737.794 5..."
3,Albiano d'Ivrea,"POLYGON ((415942.504 5033304.609, 416079.315 5..."
4,Almese,"POLYGON ((376934.962 4999073.854, 376960.555 4..."
...,...,...
7898,Villaputzu,"MULTIPOLYGON (((556602.772 4375162.650, 556598..."
7899,Villasalto,"POLYGON ((541186.146 4375105.653, 541180.146 4..."
7900,Villasimius,"MULTIPOLYGON (((546451.098 4325900.553, 546443..."
7901,Villasor,"POLYGON ((500362.475 4361229.767, 500397.474 4..."


In [9]:
df = pd.read_csv('datasetGenerator/output/routes.csv')
df

,fromPlaceName,fromPalceCord,toPlaceName,toPlaceCord,date,time,available,duration,startTime,endTime,...,walkDistance,walkLimitExceeded,generalizedCost,elevationLost,elevationGained,transfers,fare,url_to_watch,rawOutput,ResponseData
0,Camporgiano,"44.15955695,10.3330076",Careggine,"44.11944175,10.3256665",10-01-2023,1:00pm,True,4493.0,1.696246e+12,1.696251e+12,...,860.48,False,6327.0,0.0,0.0,1.0,"[{'startTime': 1696246386000, 'endTime': 16962...",http://localhost:8080/?module=planner&fromPlac...,"{'duration': 4493, 'startTime': 1696246386000,...",NaN
1,Camporgiano,"44.15955695,10.3330076",Careggine,"44.11944175,10.3256665",10-01-2023,2:00pm,True,4133.0,1.696259e+12,1.696263e+12,...,2096.95,False,6855.0,0.0,0.0,1.0,"[{'startTime': 1696259346000, 'endTime': 16962...",http://localhost:8080/?module=planner&fromPlac...,"{'duration': 4133, 'startTime': 1696259346000,...",NaN
2,Camporgiano,"44.15955695,10.3330076",Careggine,"44.11944175,10.3256665",10-01-2023,3:00pm,True,4133.0,1.696259e+12,1.696263e+12,...,2096.95,False,6855.0,0.0,0.0,1.0,"[{'startTime': 1696259346000, 'endTime': 16962...",http://localhost:8080/?module=planner&fromPlac...,"{'duration': 4133, 'startTime': 1696259346000,...",NaN
3,Camporgiano,"44.15955695,10.3330076",Careggine,"44.11944175,10.3256665",10-01-2023,4:00pm,True,4133.0,1.696259e+12,1.696263e+12,...,2096.95,False,6855.0,0.0,0.0,1.0,"[{'startTime': 1696259346000, 'endTime': 16962...",http://localhost:8080/?module=planner&fromPlac...,"{'duration': 4133, 'startTime': 1696259346000,...",NaN
4,Camporgiano,"44.15955695,10.3330076",Careggine,"44.11944175,10.3256665",10-01-2023,5:00pm,True,4133.0,1.696259e+12,1.696263e+12,...,2096.95,False,6855.0,0.0,0.0,1.0,"[{'startTime': 1696259346000, 'endTime': 16962...",http://localhost:8080/?module=planner&fromPlac...,"{'duration': 4133, 'startTime': 1696259346000,...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36475,Lucca,"43.84281,10.50238",Pescaglia,"43.9649472,10.41305245",05-07-2023,8:00am,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://localhost:8080/?module=planner&fromPlac...,NaN,"{'requestParameters': {'date': '05-07-2023', '..."
36476,Lucca,"43.84281,10.50238",Pescaglia,"43.9649472,10.41305245",05-07-2023,9:00am,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://localhost:8080/?module=planner&fromPlac...,NaN,"{'requestParameters': {'date': '05-07-2023', '..."
36477,Lucca,"43.84281,10.50238",Pescaglia,"43.9649472,10.41305245",05-07-2023,10:00am,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://localhost:8080/?module=planner&fromPlac...,NaN,"{'requestParameters': {'date': '05-07-2023', '..."
36478,Lucca,"43.84281,10.50238",Pescaglia,"43.9649472,10.41305245",05-07-2023,11:00am,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://localhost:8080/?module=planner&fromPlac...,NaN,"{'requestParameters': {'date': '05-07-2023', '..."


In [10]:
df.loc[0, 'fromPalceCord']

'44.15955695,10.3330076'

In [54]:
def plotDataset(df_routes, df_shapes, city, date, time):
    df_routes = df_routes[(df_routes['fromPlaceName'] == city) & (df_routes['date'] == date) & (df_routes['time'] == time)]
    df_routes.drop(columns=['walkDistance','walkLimitExceeded','generalizedCost','elevationLost','elevationGained','transfers','fare','url_to_watch','ResponseData','rawOutput'], inplace=True)
    if (len(df_routes) == 0):
        return (print('No trips available'))
    df_routes['duration'] = df_routes['duration'].astype(int)
    intervalli = [0, 3000, 6000, 30000, 9999999]
    colori = ['#8ceb34', '#ebc334', '#eb3a34', '#6b6a6a']
    df_routes['colore'] = pd.cut(df_routes['duration'], bins=intervalli, labels=colori)
    df = df_shapes.merge(df_routes, left_on="COMUNE", right_on="toPlaceName", how="inner")
    gdf_nuts_geom = gpd.GeoDataFrame(df,geometry='geometry')
    gdf_nuts_geom.dropna(inplace=True)
    # gdf_nuts_geom = gdf_nuts_geom[gdf_nuts_geom.cluster_id.isin(clusters)] 
    m = folium.Map((44.096563, 10.418314),zoom_start=10.4)
    folium.GeoJson(
        gdf_nuts_geom,
        style_function=lambda feature: {
            'fillColor': feature['properties']['colore'],
            'color' : feature['properties']['colore'],
            'weight' : 0.7,
            'fillOpacity' : 0.5,
            },
        tooltip=folium.GeoJsonTooltip(
            fields=['COMUNE','duration','fromPlaceName'],
            aliases=['COMUNE','duration','fromPlaceName'],
            localize=True
        )
    ).add_to(m)

    coordinate_string = df.loc[0, 'fromPalceCord']
    coordinate_list = [float(coord) for coord in coordinate_string.split(',')]


    folium.Marker(
        location=coordinate_list,
        popup="Punto di partenza",
        icon=folium.Icon(color='blue')
    ).add_to(m)

    return (m)





In [55]:
plotDataset(df, comuni_shape, 'Lucca', '10-01-2023', '11:00am')

/tmp/ipykernel_7052/246854835.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_routes.drop(columns=['walkDistance','walkLimitExceeded','generalizedCost','elevationLost','elevationGained','transfers','fare','url_to_watch','ResponseData','rawOutput'], inplace=True)
/tmp/ipykernel_7052/246854835.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_routes['duration'] = df_routes['duration'].astype(int)
/tmp/ipykernel_7052/246854835.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_